In [ ]:
import sys
#Fix problem with ros python path
sys.path.remove('/opt/ros/kinetic/lib/python2.7/site-packages')

In [ ]:
import numpy as np
import os, math
import pascal_voc_writer
from tqdm import tqdm
import cv2
import csv
from matplotlib import pyplot as plt
from CNNRobotLocalisation.Utils.utils import *
from CNNRobotLocalisation.Utils.file_utils import *
%matplotlib inline

In [ ]:
# Write defined/undefined orientation flag to Pascal VOC
assert False

path = "/media/data/LocalizationDataNew/Output/Orientation/"
csv_path = "/media/data/LocalizationDataNew/DataSwarmlab/EvalData1/"

def write_orientation_defined(f,d):
    with open(f) as file:
        content = file.read()
        content = re.sub("</truncated>\n[\s\S]*<difficult>", 
                         "</truncated>\n" + 
                         "        <pose_defined>"+str(d)+"</pose_defined>\n"+
                         "        <difficult>",content)
        with open(f,'w') as file:
            content = file.write(content)

defined_dict = {}
for f in get_recursive_file_list(path):
    video_path = f.rsplit('.avi/',1)[0]+'.avi/'
    xml = video_path + 'BoundingBoxes/' + f.rsplit('/',1)[1].replace('.jpg','.xml')
    if 'OrientationDefinedCrops' not in f:
        #write_orientation_defined(xml,1)
        continue
    video_name = f.rsplit('.avi/',1)[0].replace(path,'')
    frame = f.rsplit('/',1)[1]
    if video_name not in defined_dict: defined_dict[video_name] = {}

    if 'undefined_orientation' in f:
        write_orientation_defined(xml,0)
        defined_dict[video_name][frame] = 0
    elif 'defined_orientation' in f:
        write_orientation_defined(xml,1)
        defined_dict[video_name][frame] = 1
    else:
        print('Orientation flag not contained in folder structure: '+f)

for k in defined_dict.keys():
    csv_file = csv_path+k+'_id.csv'
    rows = []
    rows = readCSV(csv_file)
    new_rows = []
    for r in rows:
        if r[0] not in defined_dict[k].keys():
            print(k + ': ' + r[0] + ' is missing')
            new_rows.append(r+[0])
        else:
            assert len(r) == 2
            new_rows.append(r+[defined_dict[k][r[0]]])
    print(new_rows)
    writeCSV(csv_file,new_rows)

In [ ]:
def minRect(frame, frameNum):
    frameStr = "frame"+str(frameNum).zfill(6)
    kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(50,50))
    (b, g, r) = cv2.split(frame)
    ret, b = cv2.threshold(b, 240, 255, cv2.THRESH_BINARY)
    b = cv2.dilate(b,kernel1)
    
    thetas = []
    im2, contours, hierarchy = cv2.findContours(b,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) > 0:
        rect = cv2.minAreaRect(contours[0])
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        for j in range(4):
            thetas.append(math.atan2(box[(j+1)%4][0]-box[j][0],box[(j+1)%4][1]-box[j][1]) * 180 / np.pi)
        if PREPARATION_MODE:
            cv2.drawContours(frame,[box],0,(0,255,255),2)
    return centroid(contours), thetas
    
def minRectR(frame, frameNum):
    frameStr = "frame"+str(frameNum).zfill(6)
    #kernel1 = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
    kernel2 = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
    (b, g, r) = cv2.split(frame)
    ret, b = cv2.threshold(b, 200, 255, cv2.THRESH_BINARY)
    ret, r = cv2.threshold(r, 240, 255, cv2.THRESH_BINARY)
    #r = cv2.erode(r,kernel1)
    r = cv2.dilate(r,kernel2)
    
    im2, contours, hierarchy = cv2.findContours(r,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    return centroid(contours)

def regLine(frame, frameNum, threshold=100, dilate=10):
    frameStr = "frame"+str(frameNum).zfill(6)
    kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(dilate,dilate))
    g = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    ret, g = cv2.threshold(g, threshold, 255, cv2.THRESH_BINARY)
    g = cv2.dilate(g,kernel1)
    g = cv2.erode(g,kernel1)
    
    im2, contours, hierarchy = cv2.findContours(g,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) > 0:
        cnt = contours[0]
        [vx,vy,x,y] = cv2.fitLine(cnt, cv2.DIST_L2,0,0.01,0.01)
        #if PREPARATION_MODE:
        #    rows,cols,_ = frame.shape
        #    lefty = int((-x*vy/vx) + y)
        #    righty = int(((cols-x)*vy/vx)+y)
        #    cv2.line(frame,(cols-1,righty),(0,lefty),(0,255,0),2)
        #    cv2.drawContours(frame,contours,0,(0,255,255),2)
        return math.atan2(vx,vy) * 180 / np.pi
    return 0

def loadBB(f):
    data = parseXML(f)['annotation']
    obj = data['object'][0]
    return int(obj['bndbox']['xmin']), \
           int(obj['bndbox']['xmax']), \
           int(obj['bndbox']['ymin']), \
           int(obj['bndbox']['ymax'])
            
def save(f,orientation):
    with open(f) as file:
        content = file.read()
    content = re.sub("<pose>.*</pose>", "<pose>"+str(orientation)+"</pose>",content)
    with open(f,'w') as file:
        content = file.write(content)

In [ ]:
#assert False #Change dir and don't override manual changes!
path = "/media/data/LocalizationDataNew/Output/Orientation/180420_by_nl_eval_copter_C.avi/BoundingBoxes"
out_path = "/media/data/LocalizationDataNew/Output/ValRotationDebug/"
os.makedirs(out_path, exist_ok=True)
PREPARATION_MODE = False
MODE = "copter"

files = get_recursive_file_list(path,file_extensions=[".xml"])
i=0
last_ax, last_ay = 0,0
for f in tqdm(files):
    i += 1
    frame = cv2.imread(f.replace('.xml','.jpg'))
    frameStr = "frame"+str(i).zfill(6)
    #if parseXML(f)['annotation']['object'][0]['pose_defined'] == str(0):
    #    continue
    bb = loadBB(f)
    margin = 20
    frame = frame[bb[2]-margin:bb[3]+margin, bb[0]-margin:bb[1]+margin]
    ax = int( (bb[0]+bb[1])/2 )
    ay = int( (bb[2]+bb[3])/2 )
    if MODE=='copter':
        size = 100
        cx2, cy2 = minRectR(frame,i)
        (cx, cy), thetasRect = minRect(frame,i)
        theta = math.atan2(cx2-cx,cy2-cy) * 180 / np.pi
        for t in thetasRect:
            if min(abs(t-theta),360-abs(t-theta)) < 45:
                theta = t
    else:
        size = 20
        cx = int(frame.shape[1] / 2)
        cy = int(frame.shape[0] / 2)
        theta_reg = regLine(frame,i)
        theta_movement = (180+math.atan2(ax-last_ax, ay-last_ay) * 180 / np.pi) % 360
        if abs(ax-last_ax) > 8 or abs(ay-last_ay) > 8:
            theta = theta_movement
        else:
            theta = theta_reg
        last_ax, last_ay = ax, ay
      
    if PREPARATION_MODE:
        if MODE=='copter':
            cv2.circle(frame,(cx2,cy2), 5, (0,0,255), -1)
            cv2.circle(frame,(cx,cy), 5, (0,255,0), -1)
        cv2.arrowedLine(frame,
            (cx,cy),
            (int(cx-math.sin(theta*np.pi/180)*size),int(cy-math.cos(theta*np.pi/180)*size)),
            (0,0,255),2)
        file_name = out_path+frameStr+".jpg"
        cv2.imwrite(file_name, frame)
    else:
        save(f,round(theta))